# Programming Assignment 1

In [1]:
# Setup and load starter codes

import numpy as np

import data
import network

from matplotlib import pyplot as plt

np.random.seed(seed = 100) # fix random seed to make things reproducible

## Loading Data

In [2]:
train = data.load_data()
train[0].shape, train[1].shape

((60000, 784), (60000,))

In [3]:
test = data.load_data(train=False)
test[0].shape, test[1].shape

((10000, 784), (10000,))

## Data EDA

### Training

In [4]:
# Classes distribution
c_counts = np.zeros(10)
for c in train[1]:
    c_counts[c] += 1
c_counts

array([6000., 6000., 6000., 6000., 6000., 6000., 6000., 6000., 6000.,
       6000.])

### Test

In [5]:
# Classes distribution
c_counts = np.zeros(10)
for c in test[1]:
    c_counts[c] += 1
c_counts

array([1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
       1000.])

## Logistic Regression

### Pre-process
Get two sets of data for the logistic regression

In [6]:
# filter dataset before start
def set1_filter(dataset):
    # 0 or 6
    idx = []
    img, lab = dataset
    for i in range(len(lab)):
        if lab[i] == 0 or lab[i] == 6:
            idx.append(i)

    return img[idx], lab[idx]

def set2_filter(dataset):
    # 2 or 6
    idx = []
    img, lab = dataset
    for i in range(len(lab)):
        if lab[i] == 2 or lab[i] == 6:
            idx.append(i)

    return img[idx], lab[idx]

In [7]:
train_set1 = set1_filter(train)
print(train_set1[0].shape, train_set1[1].shape)
train_set2 = set2_filter(train)
print(train_set2[0].shape, train_set2[1].shape)

(12000, 784) (12000,)
(12000, 784) (12000,)


In [8]:
test_set1 = set1_filter(test)
print(test_set1[0].shape, test_set1[1].shape)
test_set2 = set2_filter(test)
print(test_set2[0].shape, test_set2[1].shape)

(2000, 784) (2000,)
(2000, 784) (2000,)


In [9]:
# put data into X and y respectively
train_X_set1, train_y_set1 = train_set1[0], train_set1[1]
train_X_set2, train_y_set2 = train_set2[0], train_set2[1]
test_X_set1, test_y_set1 = test_set1[0], test_set1[1]
test_X_set2, test_y_set2 = test_set2[0], test_set2[1]

In [10]:
# normalize X
train_X_set1 = data.z_score_normalize(train_X_set1)
train_X_set2 = data.z_score_normalize(train_X_set2)
test_X_set1 = data.z_score_normalize(test_X_set1)
test_X_set2 = data.z_score_normalize(test_X_set2)